# Машинное обучение
## Практическое задание 5. Рекомендательные системы
### Общая информация
### О задании
В этом задании мы будем практиковаться в построении рекомендательных систем: научимся считать метрики и пообучаем модели.

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tqdm

### Данные

Будем работать с датасетом  от онлайн-кинотеатра Okko [REKKO CHALLENGE](https://boosters.pro/championship/rekko_challenge/data).

Имеющиеся данные содержат несколько файлов. Рассмотрим всё, что у нас есть.

`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах:

In [75]:
catalogue = pd.read_json('catalogue.json').T

In [76]:
catalogue['element_uid'] = catalogue.index

In [77]:
catalogue.head()

,type,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,attributes,element_uid
1983,movie,"[purchase, rent, subscription]",140,1657223.396513,0.75361,39,1.119409,0.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1983
3783,movie,"[purchase, rent, subscription]",110,35565207.694893,0.766254,41,1.138604,0.654707,"[1, 26, 27, 28, 29, 7, 30, 31, 32, 10, 14, 15,...",3783
5208,movie,"[purchase, rent, subscription]",90,13270676.52431,0.765425,27,1.131807,0.592716,"[1, 38, 39, 40, 7, 41, 42, 43, 14, 15, 17, 18,...",5208
9744,movie,"[purchase, rent, subscription]",120,21749917.409823,0.757874,26,1.133525,0.654707,"[1, 47, 48, 49, 50, 51, 52, 53, 32, 42, 54, 14...",9744
1912,movie,"[purchase, rent]",110,9212963.985682,0.759566,7,1.110127,0.654707,"[1, 59, 60, 61, 62, 7, 52, 63, 10, 42, 54, 17,...",1912


 - `attributes` — мешок атрибутов;
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`);
 - `duration` — длительность фильма (для сериалов и многосерийных фильмов — длительность серии) в минутах, округлённая до десятков;
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков;
 - `type` — принимает значения `movie`, `multipart_movie` или `series`.

`transactions.csv` — список всех транзакций за определённый период времени:

In [78]:
transactions = pd.read_csv('/content/drive/MyDrive/transactions.csv',
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

In [79]:
transactions.head()

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
0,3336,5177,S,4.430518e+07,4282,0,50
1,481,593316,S,4.430518e+07,2989,0,11
2,4128,262355,S,4.430518e+07,833,0,50
3,6272,74296,S,4.430518e+07,2530,0,99
4,5543,340623,P,4.430518e+07,6282,0,50


In [80]:
transactions.shape

(9643012, 7)

 - `element_uid` — идентификатор элемента (фильма);
 - `user_uid` — идентификатор пользователя;
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке);
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке;
 - `watched_time` — число просмотренных по транзакции секунд;
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр;
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр.

`ratings.csv` содержит информацию о поставленных пользователями оценках:

In [81]:
ratings = pd.read_csv('ratings.csv')

In [82]:
ratings.head()

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07
3,359870,1364,10,4.430506e+07
4,359870,3578,9,4.430506e+07


In [83]:
ratings.shape

(438790, 4)

 - `rating` — поставленный пользователем рейтинг (от `0` до `10`).

Далее будем обучать модель на долю времени просмотра фильма. Для этого к матрице `transactions` добавим информацию о длительности фильма.

In [84]:
transactions = transactions.merge(catalogue, how='left', on='element_uid')

 В колонке `transactions['duration']` могут быть нули, так как значения округляются до ближайшей десятки. Поэтому добавим 10 ко всем значениям. И умножим все значения `transactions['duration']` на 60, чтобы перевести в секунды.

In [85]:
transactions.loc[:, 'duration'] += 10
transactions['duration']  = transactions['duration']*60

Для простоты вычислений далее будем использовать не всех пользователей.

In [86]:
transactions_small = transactions.loc[transactions.user_uid < transactions.user_uid.quantile(.01)]

In [87]:
transactions.element_uid.nunique()

8296

Чтобы оценивать качество предсказаний построенных алгоритмов, разобьем данные на обучающую и тестовую выборки. Для этого вычислим 75-й перцентиль признака `ts` и разделим данные `transactions_small` во времени, так, чтобы в обучающую выборку попало 75% данных, а в тестовую — остальные 25%.

In [88]:
transactions_test = transactions_small.loc[transactions_small.ts > transactions_small.ts.quantile(.75)].reset_index()
transactions_train = transactions_small.loc[transactions_small.ts < transactions_small.ts.quantile(.75)].reset_index()

In [89]:
transactions_train.shape

(72319, 17)

In [90]:
transactions_train.user_uid.nunique()

4498

In [91]:
transactions_test.user_uid.nunique()

2733

Данные `ratings` тоже необходимо сократить и исключить оценки из тестовой части:

In [92]:
test_user_ids = set(transactions_test[['element_uid', 'user_uid']].itertuples(index=False))
train_user_ids = set(transactions_train[['element_uid', 'user_uid']].itertuples(index=False))
drop_index = []
for row in tqdm.tqdm(ratings.itertuples()):
    if (row.element_uid, row.user_uid) in test_user_ids or not (row.element_uid, row.user_uid) in train_user_ids:
        drop_index.append(row.Index)

438790it [00:01, 294370.53it/s]


In [93]:
ratings.drop(drop_index, inplace=True)
ratings.reset_index(inplace=True)

In [94]:
ratings.shape

(2836, 5)

На выделенной обучающей выборке необходимо обучить рекомендательную систему и предсказать top-20 наиболее релевантных для пользователя идентификаторов контента.

Получим значение целевой переменной для итогового алгоритма — долю времени просмотра фильма:

In [95]:
transactions_train['target'] = transactions_train.watched_time/transactions_train.duration

In [96]:
transactions_test['target'] = transactions_test.watched_time/transactions_test.duration

In [97]:
transactions_train.head()

,index,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer,type,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,attributes,target
0,2374048,2771,850,S,4.374002e+07,1672,0,50,series,"[purchase, subscription]",3000,38554571.777868,0.712737,12,1.138604,0.654707,"[3224, 34069, 34070, 34071, 270, 15686, 123, 4...",0.557333
1,2374075,3783,1912,S,4.374001e+07,6171,4,76,movie,"[purchase, rent, subscription]",7200,35565207.694893,0.766254,41,1.138604,0.654707,"[1, 26, 27, 28, 29, 7, 30, 31, 32, 10, 14, 15,...",0.857083
2,2374197,1539,1912,S,4.373998e+07,70,4,76,movie,"[purchase, rent, subscription]",6000,41367225.524366,0.698501,25,1.141929,0.592716,"[18207, 18208, 17747, 6711, 270, 18209, 130, 3...",0.011667
3,2374319,3045,6021,R,4.373995e+07,7571,0,50,movie,"[purchase, rent]",8400,42856315.336426,0.66601,18,1.141929,0.68041,"[110, 624, 625, 626, 7, 627, 42, 54, 175, 131,...",0.90131
4,2374550,7081,5662,S,4.373989e+07,3179,0,50,movie,"[purchase, rent, subscription]",6600,39953812.47221,0.697498,19,1.140273,0.449667,"[10070, 244, 10081, 10082, 245, 10083, 10, 14,...",0.481667


В поле `target` присутствуют значения больше 1, так как для сериалов и многосерийных фильмов используется средняя длина серии — заменим в таких случаях целевую переменную на 1:

In [98]:
transactions_train.loc[transactions_train['target'] > 1, 'target'] = 1
transactions_test.loc[transactions_test['target'] > 1, 'target'] = 1

Далее в задании везде будем обозначать как $r_{ui}$ значение этой целевой переменной для пользователя $u$ и элемента $i$.

### Задание 0

Для оценки качества предсказаний будем использовать метрику $MNAP@20$, которую использовали в соревновании.
Метрика отличается от оригинальной $MAP$ тем, что значение для каждого пользователя нормализуется не на константу, а на число потреблённых фильмов. Таким образом, вес угадывания одного фильма больше у пользователей, имеющих меньшее число просмотров.


$$MNAP@20 = \dfrac{1}{|U|}\sum_{u\in U}\dfrac{1}{\min(n_u, 20)}\sum_{i=1}^{20}r_u(i)p_u@i $$
$$p_u@k = \dfrac{1}{k}\sum_{j=1}^kr_u(j)$$
- $r_{u}(i)$ — потребил ли пользователь u контент, предсказанный ему на месте i (1 либо 0);
- $n_u$ — количество элементов, которые пользователь потребил за тестовый период;
- $U$ — множество тестовых пользователей.

Реализуйте функцию `mnap_k`, вычисляющую метрику качества $MNAP@K$, где

- `predictions` — предсказания фильмов для пользователей  `List[List[int]]`;

- `target` — фильмы которые посмотрел пользователь  `List[Set[int]]`.

In [122]:
from typing import List, Set

In [100]:
def mnap_k(predictions: List[List[int]], target: List[Set[int]], k: int = 20) -> float:
    total_mnap = 0.0

    for u in range(len(predictions)):
        correct_predictions = [1 if film in target[u] else 0 for film in predictions[u][:k]]
        precision_at_k = [sum(correct_predictions[:j + 1]) / (j + 1) for j in range(k) if correct_predictions[j] == 1]

        if precision_at_k:
            total_mnap += sum(precision_at_k) / len(precision_at_k)

    return total_mnap / len(predictions)


Проведем элементарные проверки на корректность реализации:

In [101]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(1, 21)), set(np.arange(2, 22)), set(np.arange(3, 23))]
assert mnap_k(test, target) == 1.0

In [102]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(1, 11)), set(np.arange(2, 12)), set(np.arange(3, 13))]
assert mnap_k(test, target) == 1.0

In [103]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(2, 21)), set(np.arange(2, 22)), set(np.arange(3, 23))]
assert round(mnap_k(test, target), 3) == 0.954

### Задание 1 [1.5 балла]



#### Memory-based
Два пользователя похожи, если им нравятся одинаковые фильмы.
Рассмотрим двух пользователей $u$ и $v$ и обозначим через $I_{uv}$ множество фильмов $i$, которые просмотрели пользователи:
$$
    I_{uv}
    =
    \{
        i \in I
        |
        \exists r_{ui}
        \ \&\
        \exists r_{vi}
    \}.
$$
Тогда сходство двух данных пользователей можно вычислить через корреляцию Пирсона:
$$
    w_{uv}
    =
    \frac{
        \sum_{i \in I_{uv}}
            (r_{ui} - \bar r_u)
            (r_{vi} - \bar r_v)
    }{
        \sqrt{
        \sum_{i \in I_{uv}}
            (r_{ui} - \bar r_u)^2
        }
        \sqrt{
        \sum_{i \in I_{uv}}
            (r_{vi} - \bar r_v)^2
        }
    },
$$
где $\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей.

Чтобы вычислять сходства между элементами $i$ и $j$, введём множество пользователей $U_{ij}$, для которых известны целевые переменные для этих элементов:
$$
    U_{ij}
    =
    \{
        u \in U
        |
        \exists r_{ui}
        \ \&\
        \exists r_{uj}
    \}.
$$
Тогда сходство двух данных товаров можно вычислить через корреляцию Пирсона:
$$
    w_{ij}
    =
    \frac{
        \sum_{u \in U_{ij}}
            (r_{ui} - \bar r_i)
            (r_{uj} - \bar r_j)
    }{
        \sqrt{
        \sum_{u \in U_{ij}}
            (r_{ui} - \bar r_i)^2
        }
        \sqrt{
        \sum_{u \in U_{ij}}
            (r_{uj} - \bar r_j)^2
        }
    },
$$
где $\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов

Реализуйте класс `MemoryBased` с методами:

- `__init__` — конструктор класса, принимает на вход матрицу целевых переменных $r_{ui}$;
- `user_similarity` — возвращает вектор $w_u$ сходства пользователя $u$ и всех пользователей из обучающей выборки; если пользователь $u$ не встречался в обучающей выборке, то возвращайте нулевой вектор;
- `item_similarity` — возвращает вектор $w_i$ сходства фильма $i$ и всех фильмов из обучающей выборки; если фильм $i$ не встречался в обучающей выборке, то возвращайте нулевой вектор.



Уделите особое внимание оптимизации кода. Используйте векторизацию и матричные вычисления, где это возможно.

In [104]:
class MemoryBased:
    def __init__(self, ratings):
        self.ratings = ratings
        self.num_users, self.num_items = ratings.shape

    def user_similarity(self, test_user):
        centered_user_ratings = self.ratings - np.nanmean(self.ratings, axis=1)[:, np.newaxis]
        centered_test_ratings = test_user - np.nanmean(test_user)

        dot_product = np.dot(centered_user_ratings, centered_test_ratings)
        norm_user_ratings = np.linalg.norm(centered_user_ratings, axis=1)
        norm_test_ratings = np.linalg.norm(centered_test_ratings)

        denominator = np.sqrt(np.multiply(norm_user_ratings**2, norm_test_ratings**2) + 1e-8)
        similarity_vector = np.divide(dot_product, denominator)
        similarity_vector[np.isnan(similarity_vector)] = 0
        return similarity_vector

    def item_similarity(self, test_item):

        centered_item_ratings = self.ratings - np.nanmean(self.ratings, axis=0)[np.newaxis, :]
        centered_test_ratings = test_item - np.nanmean(test_item)
        dot_product = np.dot(centered_item_ratings.T, centered_test_ratings)
        norm_item_ratings = np.linalg.norm(centered_item_ratings, axis=0)
        norm_test_ratings = np.linalg.norm(centered_test_ratings)

        denominator = np.outer(norm_item_ratings, norm_test_ratings) + 1e-8

        similarity_vector = np.divide(dot_product, denominator)
        similarity_vector[np.isnan(similarity_vector)] = 0

        return similarity_vector

Проведем элементарные проверки на корректность реализации:

если считаем, что

$\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей, посчитанная по всем просмотренным фильмам

$\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов, посчитанная по всем пользователям, которые смотрели фильм

In [105]:
I = np.array([[0.5, 0.4, 0, 0.1],
              [0, 0, 0.4, 0.5],
              [0.4, 0.4, 0.1, 0],
              [0.5, 0.4, 0.1, 0.1]])
user_based = MemoryBased(I)
result = user_based.user_similarity(np.array([0.5, 0.4, 0, 0.1]))
assert np.all(np.round(result, 2) == np.array([1.0, -0.93, 0.92, 0.98]))

если считаем, что

$\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей, посчитанная по множеству $I_{uv}$

$\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов, посчитанная по множеству $I_{ij}$

In [106]:
I = np.array([[0.5, 0.4, 0, 0.1],
         [0, 0.1, 0.5, 0.4],
         [0.5, 0.5, 0.5, 0.5],
         [0.5, 0.4, 0, 0.1]])
user_based = MemoryBased(I)
result = user_based.user_similarity(np.array([0.5, 0.4, 0, 0.1]))
assert np.all(np.round(result, 2) == np.array([1.0, -1.0, 0.0, 1.0]))

### Задание 2 [1 балл]

#### User-Based
В подходе на основе сходств пользователей
определяется множество $U(u_0)$ пользователей, похожих на данного:
$$
    U(u_0)
    =
    \{v \in U
        |
        w_{u_0 v} > \alpha
    \}.
$$
После этого для каждого товара вычисляется, как часто он покупался пользователями из $U(u_0)$:
$$
    p_{i}
    =
    \frac{
        |\{u \in U(u_0) | \exists r_{ui}\}|
    }{
        |U(u_0)|
    }.
$$
Пользователю рекомендуются $k$ товаров с наибольшими значениями $p_i$.
Данный подход позволяет строить рекомендации, если для данного пользователя найдутся похожие.
Если же пользователь является нетипичным, то подобрать что-либо не получится.

Реализуйте класс `UserBased`, наследованный от `MemoryBased`. Считайте, что $\alpha = 0$.

Класс `UserBased` должен иметь метод реализованный `recomendation_k` с параметром k, возвращающий матрицу предсказаний размера N x k (для каждого пользователя из user_vectors предсказывать top-k фильмов).

In [107]:
class UserBased(MemoryBased):
    def recommendation_k(self, user_vectors, k=20):
        num_users, num_items = user_vectors.shape
        recommendations = np.zeros((num_users, k), dtype=int)

        for u in range(num_users):
            similarities = self.user_similarity(user_vectors[u])

            # Исключаем самого пользователя из рассмотрения
            similarities[u] = -1

            top_similar_users = np.argsort(similarities)[::-1][:k]
            item_counts = np.sum(self.ratings[top_similar_users] > 0, axis=0)
            top_items = np.argsort(item_counts)[::-1]

            # Записываем рекомендации в матрицу
            recommendations[u] = top_items[:k]

        return recommendations


Проведем элементарные проверки на корректность реализации:

In [108]:
I = np.array([[0.5, 0.4, 0, 0.1],
              [0, 0.1, 0.2, 0.5],
              [0.5, 0.5, 0.4, 0],
              [0.5, 0.4, 0.5, 0.1]])

user_based = UserBased(I)
result = user_based.recommendation_k(np.array([[0.5, 0.4, 0, 0.1]]), k=1)
result
assert np.all(result == np.array([[2]]))

### Задание 3 [1 балл]

#### Item-Based

Определяется множество фильмов, похожих на те, которые интересовали данного пользователя:
$$
    I(u_0)
    =
    \{
        i \in I
        |
        \exists r_{u_0 i_0},
        w_{i_0 i} > \alpha
    \}.
$$
Затем для каждого товара из этого множества вычисляется его сходство с пользователем:
$$
    p_i
    =
    \max_{i_0: \exists r_{u_0 i_0}}
    w_{i_0 i}.
$$
Пользователю рекомендуются $k$ фильмов с наибольшими значениями $p_i$.


Реализуйте класс `ItemBased`, наследованный от `MemoryBased`.

Класс `ItemBased` должен иметь метод реализованный `recomendation_k` с параметром k, возвращающий матрицу предсказаний размера N x k (для каждого пользователя из user_vectors предсказывать top-k фильмов).

In [109]:
class ItemBased(MemoryBased):
    def recommendation_k(self, user_vectors, k=20):
        num_users, num_items = user_vectors.shape
        recommendations = np.zeros((num_users, k), dtype=int)

        for u in range(num_users):
            user_ratings = user_vectors[u]
            candidate_items = np.where(user_ratings == 0)[0]
            item_similarities = np.max(self.item_similarity(self.ratings[:, candidate_items]), axis=0)
            top_items = np.argsort(item_similarities)[::-1][:k]

            recommendations[u] = candidate_items[top_items]

        return recommendations



Проведем элементарные проверки на корректность реализации:

In [ ]:
I = np.array([[0.5, 0.4, 0, 0.1],
         [0, 0.1, 0.2, 0.5],
         [0.5, 0.5, 0.4, 0],
         [0.5, 0.4, 0.5, 0.1]])
item_based = UserBased(I)
result = item_based.recomendation_k(np.array([0.5, 0.4, 0, 0.1]), k=1)
assert np.all(result == np.array([2]))

### Задание 4 [1 балл]

Сформируйте матрицы $R = \{r_{ui}\}_{u \in U, \, i \in I}$ по данным `transactions_train` и `transactions_test`, где $r_{ui}$ — доля времени взаимодействовия пользователя с фильмом.

С помощью Item-Based и User-Based подходов получите предсказания для пользователей из тестовой выборки и оцените качество с помощью реализованной в задании 0 функции `mnap_k` при `k=20`.

In [110]:
# Создаем матрицу R для обучающей выборки с учетом 'target'
R_train = pd.pivot_table(transactions_train, values='target', index='user_uid', columns='element_uid', aggfunc=np.sum, fill_value=0).values

# Создаем матрицу R для тестовой выборки с учетом 'target'
R_test = pd.pivot_table(transactions_test, values='target', index='user_uid', columns='element_uid', aggfunc=np.sum, fill_value=0).values

In [130]:
print(R_train.shape)
print(R_test.shape)

(4498, 5124)
(2733, 3578)


In [114]:
item_based = ItemBased(R_train)
user_based = UserBased(R_train)

In [115]:
user_based_recommendations = user_based.recommendation_k(R_train, k=20)

In [ ]:
item_based_recommendations = item_based.recommendation_k(R_train, k=20)

In [155]:
user_based_predictions = user_based_recommendations[:, :R_test.shape[1]]
item_based_predictions = item_based_recommendations[:, :R_test.shape[1]]

In [156]:
common_users = set(transactions_test['user_uid']).intersection(set(transactions_train['user_uid']))
common_users_idx = [idx for idx, user in enumerate(transactions_train['user_uid']) if user in common_users]

In [157]:
common_users_idx = [idx for idx in common_users_idx if idx < len(user_based_recommendations)]

user_based_predictions = [set(user_based_recommendations[u]) for u in common_users_idx if u < len(user_based_recommendations)]
item_based_predictions = [set(item_based_recommendations[u]) for u in common_users_idx if u < len(item_based_recommendations)]


In [184]:
true_values = [set(np.where(R_test[u] > 0)[0]) for u in common_users_idx if u < R_test.shape[0]]

user_based_quality = mnap_k(user_based_predictions, true_values, k=20)
item_based_quality = mnap_k(item_based_predictions, true_values, k=20)

In [186]:
print("User-Based Quality:", user_based_quality)
print("Item-Based Quality:", item_based_quality)

User-Based Quality: 0.7845181179385476
Item-Based Quality: 0.7143579610497344


### Задание 5 [3 балла]

#### Latent factor model (LFM)

В этом подходе значение целовой переменной $r_{ui}$ пользователя $u$, поставленная фильму $i$, ищется как скалярное произведение векторов $p_u$ и $q_i$ в некотором пространстве $R^K$ латентных признаков:
$$
    r_{ui}
    \approx
    \langle p_u, q_i \rangle.
$$



Иными словами, модель находит пространство признаков, в котором мы описываем и фильмы, и пользователей и в котором целевая переменная является мерой близости между фильмами и пользователями.

Для настройки модели будем минимизировать следующий функционал:

\begin{equation}
\label{eq:lfmReg}
    \sum_{(u, i) \in R}
        \left(
            r_{ui}
            - \langle p_u, q_i \rangle
        \right)^2
    +
    \lambda
    \sum_{u \in U}
        \|p_u\|^2
    +
    \mu
    \sum_{i \in I}
        \|q_i\|^2
    \to
    \min_{P, Q}
\end{equation}

В [статье](https://dl.acm.org/doi/10.1145/1864708.1864726) описан метод оптимизации ALS (Alternating Least Squares) для данного функционала.
В методе проводятся $N$ итераций, в рамках каждой итерации сначала оптимизируется $p$ при фиксированном
$q$, затем $q$ при фиксированном $p$.

Составим матрицу $P$ из векторов $p_u$ и матрицу $Q$ из векторов $q_i$. Матрицей $Q[u] \in R^{n_u×K}$ будем обозначать подматрицу матрицы $Q$ только для товаров, оцененных пользователем $u$, где $n_u$ – количество оценок пользователя $u$.
Шаг перенастройки $p_u$ при фиксированной матрице $Q$ сводится к настройке Ridge-регрессии и выглядит так:
$$A_u = Q[u]^T Q[u] $$
$$d_u = Q[u]^Tr_u $$
$$p_u = (\lambda n_uI + A_u)^{−1}d_u
$$

Выпишите формулы для перенастройки $q_i$ при фиксированной матрице $P$.

**Ответ:**

Реализуйте функцию `latent_factor`, которая для каждого пользователя из списка `test` возвращает top-k идентификаторов контента (`element_uid`).

Для тестирования матрицы `P` и `Q` задайте случайными `0.1 * np.random.random(...)`.

Исследуйте качество и время работы в зависимости от размерности $K$
пространства латентных признаков. Ведет ли увеличение $K$ к переобучению?


In [192]:
from scipy.optimize import minimize

In [187]:
lambda_p = 0.2
mu = 0.001
N = 20
K = 10

In [188]:
train = transactions_train[['user_uid', 'element_uid']]
target = transactions_train['target']
test = transactions_test['user_uid'].unique()

In [ ]:
def latent_factor(train, target, test, lambda_, mu, N, K, P, Q, k = 20):
    #╰( ͡° ͜ʖ ͡° )つ──☆*:・
    pass

In [243]:
def latent_factor(train, target, test, lambda_, mu, N, K, P, Q, k=20):
    user_index_map = {user: idx for idx, user in enumerate(train['user_uid'].unique())}
    item_index_map = {item: idx for idx, item in enumerate(train['element_uid'].unique())}

    num_users = len(user_index_map)
    num_items = len(item_index_map)
    R = np.zeros((num_users, num_items))

    for _, row in train.iterrows():
        user_idx = user_index_map[row['user_uid']]
        item_idx = item_index_map[row['element_uid']]
        R[user_idx, item_idx] = target[user_idx]

    P_optimized, Q_optimized = train_lfm(R, K, lambda_, mu)
    recommendations = []

    for user in test:
        user_idx = user_index_map[user]
        predicted_ratings = np.dot(P_optimized[user_idx, :], Q_optimized.T)
        top_indices = np.argsort(predicted_ratings)[::-1][:k]
        top_items = [item for item, idx in item_index_map.items() if idx == top_indices[0]]
        recommendations.append(top_items)

    return np.array(recommendations)

In [251]:
train = pd.DataFrame({'user_uid': [1, 1, 2, 2], 'element_uid': [1, 2, 3, 4]})
target =  np.array( [0.1, 0.8, 0.2, 0.3 ])
test = np.array([1, 2])
lambda_p = 0.2
us = train.loc[:,'user_uid'].astype('int')
mov = train.loc[:,'element_uid'].astype('int')
U = np.unique(us)
I = np.unique(mov)
Q = 0.1 * np.ones((I.max(), K))
P = 0.1 * np.ones((U.max(), K))

assert np.all(latent_factor(train, target, test, lambda_p, mu, N, K, P, Q, k = 20) == np.array([[4], [4]]))

Оцените  качество разботы функции `latent_factor` на тестовой выборке по метрике `mnap_k` при `k=20`.

### Задание 6 [2 балла]

#### Content-based

В соревновании осталось много неиспользуемых нами данных.
Сформируйте из них признаковое описание пары (пользователь, фильм), придумав минимум 10 содержательных (!) признаков. Целевой переменной, как и раньше, будет являться доля времени взаимодействия пользователя и фильма. Соберите данные, обучите на полученных данных линейную регрессию и градиентный бустинг.
Для каждого пользователя из тестовой выборки предскажите 20 фильмов, с максимальной долей времени взаимодействия. Оцените качество предсказания с помощью функции `mnap_k` при `k=20`. Какого качество удалось достичь?